In [2]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import portfolioopt as pfopt
import cvxopt as opt

In [3]:
#http://www.learndatasci.com/python-finance-part-yahoo-finance-api-pandas-matplotlib/
#from pandas_datareader import data



# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['SPY', 'VGK', 'VWO', 'VNQ','BND']

# Define which online source one should use
data_source = 'yahoo'

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2010-01-01'
end_date = '2017-12-10'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_data = web.DataReader(tickers, data_source, start_date, end_date)

# Getting just the adjusted closing prices. This will return a Pandas DataFrame
# The index in this DataFrame is the major index of the panel_data.
close = panel_data.ix['Close']

# Getting all weekdays between 01/01/2000 and 12/31/2016
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

# How do we align the existing prices in adj_close with our new set of dates?
# All we need to do is reindex close using all_weekdays as the new index
close = close.reindex(all_weekdays)

close.head(10)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:21: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,BND,SPY,VGK,VNQ,VWO
2010-01-01,NaN,NaN,NaN,NaN,NaN
2010-01-04,78.680000,113.330002,49.959999,44.549999,42.400002
2010-01-05,78.910004,113.629997,49.880001,44.500000,42.540001
2010-01-06,78.879997,113.709999,50.180000,44.419998,42.799999
2010-01-07,78.820000,114.190002,49.860001,44.900002,42.430000
2010-01-08,78.900002,114.570000,50.360001,44.570000,42.689999
2010-01-11,78.930000,114.730003,50.759998,44.830002,42.750000
2010-01-12,79.230003,113.660004,50.020000,44.080002,42.130001
2010-01-13,79.050003,114.620003,50.700001,44.970001,42.299999
2010-01-14,79.239998,114.930000,50.599998,44.930000,42.200001


In [4]:
returnsclose = close.pct_change()
returncloseclean = returnsclose.dropna(how='any') 
returncloseclean 

avg_rets = returncloseclean.mean()
cov_mat = returncloseclean.cov()
avg_rets, cov_mat


(BND    0.000021
 SPY    0.000469
 VGK    0.000176
 VNQ    0.000387
 VWO    0.000110
 dtype: float64,           BND       SPY       VGK       VNQ       VWO
 BND  0.000005 -0.000007 -0.000008 -0.000001 -0.000006
 SPY -0.000007  0.000086  0.000110  0.000082  0.000103
 VGK -0.000008  0.000110  0.000194  0.000109  0.000155
 VNQ -0.000001  0.000082  0.000109  0.000139  0.000105
 VWO -0.000006  0.000103  0.000155  0.000105  0.000179)

In [5]:
weights = pfopt.tangency_portfolio(cov_mat, avg_rets)

In [6]:
weights

BND    6.779586e-01
SPY    3.220414e-01
VGK    1.490627e-09
VNQ    1.998189e-08
VWO    1.452104e-09
dtype: float64

In [22]:
weights = pfopt.markowitz_portfolio(cov_mat, avg_rets, .01, allow_short=True, market_neutral=False)
weights

BND    -9.204035
SPY    26.402271
VGK    -8.164522
VNQ     0.425621
VWO    -8.459334
dtype: float64

In [3]:
import numpy as np
n_assets = 4

## NUMBER OF OBSERVATIONS
n_obs = 1000

return_vec = np.random.randn(n_assets, n_obs)

In [5]:
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd


In [ ]:
def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(wt), returns, risks

weights, returns, risks = optimal_portfolio(return_vec)

plt.plot(stds, means, 'o')
plt.ylabel('mean')
plt.xlabel('std')
plt.plot(risks, returns, 'y-o')